# CSC411 Assignment 3
Let's get it.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

## P1: Reading Data

In [3]:
fake_data = pd.read_fwf("./data/clean_fake.txt", header=None)
real_data = pd.read_fwf("./data/clean_real.txt", header=None)

In [4]:
fake_train, fake_test = train_test_split(fake_data, test_size=0.15)
real_train, real_test = train_test_split(real_data, test_size=0.15)

fake_train, fake_validation = train_test_split(fake_train, test_size=0.15/0.85)
real_train, real_validation = train_test_split(real_train, test_size=0.15/0.85)

## P2: Naive Bayes

In [11]:
def generate_counts(data):
    '''
    data is an array of strings
    '''
    counts = {}
    for line in data:
        words = line.split(" ")
        for word in words:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    return counts

In [27]:
fake_counts = generate_counts(fake_train[0])
real_counts = generate_counts(real_train[0])

for key in fake_counts.keys():
    if key not in real_counts:
        real_counts[key] = 0
for key in real_counts.keys():
    if key not in fake_counts:
        fake_counts[key] = 0

total_counts = {}
for key in real_counts.keys():
    total_counts[key] = real_counts[key] + fake_counts[key]

num_fake = len(fake_train)
num_real = len(real_train)
num_total = num_fake + num_real

P_real = num_real / num_total
P_fake = num_fake / num_total

fake_freqs = {n: fake_counts[n] / num_fake for n in fake_counts.keys()}      #gives P(x | fake)
real_freqs = {n: real_counts[n] / num_real for n in real_counts.keys()}      #gives P(x | real)
total_freqs = {n: total_counts[n] / num_total for n in total_counts.keys()}  #gives P(x)

In [32]:
def make_prediction(line):
    '''
    line is a string
    '''
    words = line.split(" ")
    P_words_given_real = 1
    P_words_given_fake = 1
    for key in real_freqs:
        if key in words:
            P_words_given_real *= real_freqs[key]
            P_words_given_fake *= fake_freqs[key]
        else:
            P_words_given_real *= 1 - real_freqs[key]
            P_words_given_fake *= 1 - fake_freqs[key]
            
    numerator = P_words_given_real * P_real
    print(numerator)
    denominator = numerator + P_words_given_fake * P_fake
    #print(numerator)
    #print(denominator)
    return numerator / denominator

In [33]:
print(make_prediction("hillary donald wow united marines sessions"))

0.0


ZeroDivisionError: float division by zero